<a href="https://colab.research.google.com/github/sanaaria/Ann/blob/main/Ann_HW04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from torchvision import datasets,transforms


In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

پردازش و ترنسفورم کردن دیتا و دانلود دیتا و دادن  آن به دیتا لودر

In [27]:
transform_train = transforms.Compose([transforms.Resize((32,32)), #resises the image so it can be perfect for our model.
transforms.RandomHorizontalFlip(), # FLips the image w.r.t horizontal axis
transforms.RandomRotation(10), #Rotates the image to a specified angel
transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angle
transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Set the color params
transforms.ToTensor(), # comvert the image to tensor so that it can work with torch
transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #Normalize all the images
])
transform = transforms.Compose([transforms.Resize((32,32)),
transforms.ToTensor(),
transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
training_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train) # Data augmentation is only done on t
validation_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=100, shuffle=True) # Batch size of 100 i.e to work with 100 images
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size =100, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [28]:
dataiter = iter(training_loader)
images, labels = next(dataiter)
train_dataset=images[:1000]
data_iter_val = iter(validation_loader)
images, labels = next(data_iter_val)
valid_dataset=images[:1000]
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=2 ,shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=2,shuffle=True)

ساخت تابعی برای میانگین گرفتن از عکس ها

In [29]:
def mean(img1,img2):
  return (img1+img2)/2

نوشتن کلاس AE

In [33]:
import torch.nn as nn
import torch.nn.functional as F
class AE(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder = nn.Sequential(
    nn.Conv2d(3, 32, 4, stride=1, padding=1), #(32 , 29 , 29)
    nn.ReLU(),
    nn.Conv2d(32, 64, 4, stride=1, padding=1), #(64 , 27 , 27)
    nn.ReLU(),
    nn.Conv2d(64, 128, 4, stride=1, padding=1))#(128 , 24 , 24)
    nn.ReLU(),
    nn.Conv2d(128, 128, 4, stride=2, padding=1)#(128 , 11 , 11)))
    self.decoder1 = nn.Sequential(
    nn.ConvTranspose2d(128, 64, 3, stride=1),
    nn.LeakyReLU(),
    nn.ConvTranspose2d(64, 3, 3, stride=2), # (3,63,63)
    nn.UpsamplingNearest2d(scale_factor=32/63)
    )
    self.decoder2 = nn.Sequential(
    nn.ConvTranspose2d(128, 64, 3, stride=1),
    nn.LeakyReLU(),
    nn.ConvTranspose2d(64, 3, 3, stride=2), # (3,63,63)
    nn.UpsamplingNearest2d(scale_factor=32/63))
  def forward(self, x):
    
      coding = self.encoder(x)
      return self.decoder1(coding),self.decoder2(coding)

AE رو به مدل میدیم و لاس و اپتیمایزر رو تعریف میکنیم

In [31]:
Model = AE()
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(Model.parameters(),lr=0.0003)

مدل رو اموزش میدیم و لاس رو محاسبه میکنیم

In [32]:
n_total_steps = len(train_loader)
print(n_total_steps)
Model.train()
for epoch in range(10):
  for i,images in enumerate(train_loader):
      batch_size=images.shape[0]
      meanimage = mean(images[:int(batch_size/2)],images[int(batch_size/2):batch_size])
      
      out1,out2 = Model(meanimage)
      loss1 = criterion(out1, images[:int(batch_size/2)])
      loss2 = criterion(out2,images[int(batch_size/2):batch_size])
      loss=loss1+loss2
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if (i+1) % n_total_steps == 0:
        print(str(epoch+1) + " epoch loss is" + str(loss.item()))
        with torch.no_grad():
          for j,images in enumerate(valid_loader):
              images = images
              batch_size=images.shape[0]
              meanimage = mean(images[:int(batch_size/2)],images[int(batch_size/2):batch_size])
              out1,out2 = Model(meanimage)
              loss1 = criterion(out1, images[:int(batch_size/2)])
              loss2 = criterion(out2,images[int(batch_size/2):batch_size])
              loss=loss1+loss2
              if (j+1) % n_total_steps == 0:
                    print(str(epoch+1 ) + " epoch loss in valid set is" + str(loss.item()))





50
1 epoch loss is1.0035861730575562
1 epoch loss in valid set is0.7325537800788879
2 epoch loss is0.7326023578643799
2 epoch loss in valid set is0.581794261932373
3 epoch loss is0.5817581415176392
3 epoch loss in valid set is0.7769557237625122
4 epoch loss is0.4572448134422302
4 epoch loss in valid set is0.5315554141998291
5 epoch loss is0.5120267868041992
5 epoch loss in valid set is0.7711190581321716
6 epoch loss is0.58022141456604
6 epoch loss in valid set is0.4603534936904907
7 epoch loss is0.7877572178840637
7 epoch loss in valid set is0.4759162664413452
8 epoch loss is0.5571469068527222
8 epoch loss in valid set is0.6216781735420227
9 epoch loss is0.629176139831543
9 epoch loss in valid set is0.6638098955154419
10 epoch loss is0.7662513256072998
10 epoch loss in valid set is0.5260406732559204
